In [1]:
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import pandas as pd

def get_thrust_available(alt):
    """Calculate thrust available (N) based on altitude (m) using piecewise linear interpolation."""
    if alt <= 0:
        return 50000
    elif alt <= 2000:
        return 50000 - (alt / 2000.0) * 10000
    elif alt <= 5000:
        return 40000 - ((alt - 2000) / 3000.0) * 15000
    elif alt <= 8000:
        return 25000 - ((alt - 5000) / 3000.0) * 15000
    elif alt <= 10000:
        return 10000 - ((alt - 8000) / 2000.0) * 5000
    else:
        return 5000

def get_power_available(alt):
    """Calculate power available (MW) based on altitude (m) using piecewise linear interpolation."""
    if alt <= 0:
        return 20
    elif alt <= 2000:
        return 20 - (alt / 2000.0) * 4
    elif alt <= 5000:
        return 16 - ((alt - 2000) / 3000.0) * 6
    elif alt <= 8000:
        return 10 - ((alt - 5000) / 3000.0) * 6
    elif alt <= 10000:
        return 4 - ((alt - 8000) / 2000.0) * 2
    else:
        return 2

def get_thrust_required(alt):
    """Calculate thrust required (N) based on altitude (m) using piecewise linear interpolation."""
    if alt <= 0:
        return 10000
    elif alt <= 2000:
        return 10000 - (alt / 2000.0) * 200
    elif alt <= 5000:
        return 9800 - ((alt - 2000) / 3000.0) * 300
    elif alt <= 8000:
        return 9500 - ((alt - 5000) / 3000.0) * 200
    elif alt <= 10000:
        return 9300 - ((alt - 8000) / 2000.0) * 100
    else:
        return 9200

def get_power_required(alt):
    """Calculate power required (MW) based on altitude (m) using piecewise linear interpolation."""
    if alt <= 0:
        return 4
    elif alt <= 2000:
        return 4 - (alt / 2000.0) * 0.1
    elif alt <= 5000:
        return 3.9 - ((alt - 2000) / 3000.0) * 0.1
    elif alt <= 8000:
        return 3.8 - ((alt - 5000) / 3000.0) * 0.1
    elif alt <= 10000:
        return 3.7 - ((alt - 8000) / 2000.0) * 0.1
    else:
        return 3.6

def calculate_performance(altitude):
    """Calculate all performance metrics for a given altitude."""
    ta = get_thrust_available(altitude)
    pa = get_power_available(altitude)
    tr = get_thrust_required(altitude)
    pr = get_power_required(altitude)
    et = ta - tr
    ep = pa - pr
    return {
        'Altitude (m)': altitude,
        'Thrust Available (N)': ta,
        'Power Available (MW)': pa,
        'Thrust Required (N)': tr,
        'Power Required (MW)': pr,
        'Excess Thrust (N)': et,
        'Excess Power (MW)': ep
    }

# Widgets
altitude_input_single = widgets.FloatText(
    description='Enter Altitude (meters):',
    value=0.0,
    step=100,
    disabled=False
)
multiple_altitudes_input = widgets.Textarea(
    description='Enter Altitudes (comma-separated):',
    placeholder='e.g., 0, 1000, 5000',
    disabled=False
)
show_excel_single_checkbox = widgets.Checkbox(
    description='Show as Table?',
    value=False,
    disabled=False
)
show_excel_multiple_checkbox = widgets.Checkbox(
    description='Show as Table?',
    value=False,
    disabled=False
)
predict_single_button = widgets.Button(description="Predict for Single Altitude")
predict_multiple_button = widgets.Button(description="Predict for Multiple Altitudes")
output_area = widgets.Output()
plot_output = widgets.Output()

def predict_single(b):
    with output_area:
        output_area.clear_output()
        altitude = altitude_input_single.value
        if not 0 <= altitude <= 12000:
            print("Altitude must be between 0 and 12000 meters.")
            return
        results = calculate_performance(altitude)
        print("Results:")
        for key, value in results.items():
            print(f"{key}: {value:.1f}")
        if show_excel_single_checkbox.value:
            df = pd.DataFrame([results])
            display(df)
    with plot_output:
        plot_output.clear_output()
        alt_plot = np.linspace(0, 12000, 100)
        ta_plot = np.array([get_thrust_available(a) for a in alt_plot])
        pa_plot = np.array([get_power_available(a) for a in alt_plot])
        tr_plot = np.array([get_thrust_required(a) for a in alt_plot])
        pr_plot = np.array([get_power_required(a) for a in alt_plot])
        et_plot = ta_plot - tr_plot
        ep_plot = pa_plot - pr_plot

        fig, axs = plt.subplots(1, 2, figsize=(14, 6))
        fig.suptitle(f'Performance vs. Altitude (Current Altitude: {altitude:.0f} m)', fontsize=16)

        axs[0].plot(alt_plot, ta_plot, label='Thrust Available (N)')
        axs[0].plot(alt_plot, tr_plot, label='Thrust Required (N)')
        axs[0].plot(alt_plot, et_plot, label='Excess Thrust (N)')
        axs[0].scatter([0, 2000, 5000, 8000, 10000], [50000, 40000, 25000, 10000, 5000], color='red', marker='o', label='Data Points')
        axs[0].axvline(altitude, color='gray', linestyle='--', label=f'Current Altitude')
        axs[0].set_xlabel('Altitude (m)')
        axs[0].set_ylabel('Thrust (N)')
        axs[0].legend()
        axs[0].grid(True)
        axs[0].axhline(0, color='black', linestyle='-', linewidth=0.5)

        axs[1].plot(alt_plot, pa_plot, label='Power Available (MW)')
        axs[1].plot(alt_plot, pr_plot, label='Power Required (MW)')
        axs[1].plot(alt_plot, ep_plot, label='Excess Power (MW)')
        axs[1].scatter([0, 2000, 5000, 8000, 10000], [20, 16, 10, 4, 2], color='red', marker='o', label='Data Points')
        axs[1].axvline(altitude, color='gray', linestyle='--', label=f'Current Altitude')
        axs[1].set_xlabel('Altitude (m)')
        axs[1].set_ylabel('Power (MW)')
        axs[1].legend()
        axs[1].grid(True)
        axs[1].axhline(0, color='black', linestyle='-', linewidth=0.5)

        plt.tight_layout(rect=[0, 0.03, 1, 0.95])
        plt.show()

def predict_multiple(b):
    with output_area:
        output_area.clear_output()
        altitudes_str = multiple_altitudes_input.value.strip()
        if not altitudes_str:
            print("Please enter altitudes.")
            return
        try:
            altitudes = [float(alt.strip()) for alt in altitudes_str.split(',')]
            altitudes = [alt for alt in altitudes if 0 <= alt <= 12000]
            if not altitudes:
                print("All altitudes must be between 0 and 12000 meters.")
                return
            all_results = [calculate_performance(alt) for alt in altitudes]
            print("Results:")
            df = pd.DataFrame(all_results)
            display(df)
        except ValueError:
            print("Please enter altitudes as comma-separated numbers.")
    with plot_output:
        plot_output.clear_output()
        if altitudes_str:
            altitudes = [float(alt.strip()) for alt in altitudes_str.split(',') if 0 <= float(alt.strip()) <= 12000]
            alt_plot = np.linspace(0, 12000, 100)
            ta_plot = np.array([get_thrust_available(a) for a in alt_plot])
            pa_plot = np.array([get_power_available(a) for a in alt_plot])
            tr_plot = np.array([get_thrust_required(a) for a in alt_plot])
            pr_plot = np.array([get_power_required(a) for a in alt_plot])

            fig, axs = plt.subplots(1, 2, figsize=(14, 6))
            fig.suptitle(f'Performance for Multiple Altitudes', fontsize=16)

            axs[0].plot(alt_plot, ta_plot, label='Thrust Available (N)')
            axs[0].plot(alt_plot, tr_plot, label='Thrust Required (N)')
            axs[0].scatter(altitudes, [get_thrust_available(a) for a in altitudes], color='blue', marker='o', label='Input Altitudes')
            axs[0].set_xlabel('Altitude (m)')
            axs[0].set_ylabel('Thrust (N)')
            axs[0].legend()
            axs[0].grid(True)
            axs[0].axhline(0, color='black', linestyle='-', linewidth=0.5)

            axs[1].plot(alt_plot, pa_plot, label='Power Available (MW)')
            axs[1].plot(alt_plot, pr_plot, label='Power Required (MW)')
            axs[1].scatter(altitudes, [get_power_available(a) for a in altitudes], color='blue', marker='o', label='Input Altitudes')
            axs[1].set_xlabel('Altitude (m)')
            axs[1].set_ylabel('Power (MW)')
            axs[1].legend()
            axs[1].grid(True)
            axs[1].axhline(0, color='black', linestyle='-', linewidth=0.5)

            plt.tight_layout(rect=[0, 0.03, 1, 0.95])
            plt.show()

predict_single_button.on_click(predict_single)
predict_multiple_button.on_click(predict_multiple)

display(widgets.Label("For a single altitude:"))
display(altitude_input_single)
display(show_excel_single_checkbox)
display(predict_single_button)
display(widgets.Label("\nFor multiple altitudes (comma-separated):"))
display(multiple_altitudes_input)
display(show_excel_multiple_checkbox)
display(predict_multiple_button)
display(output_area)
display(plot_output)

Label(value='For a single altitude:')

FloatText(value=0.0, description='Enter Altitude (meters):', step=100.0)

Checkbox(value=False, description='Show as Table?')

Button(description='Predict for Single Altitude', style=ButtonStyle())

Label(value='\nFor multiple altitudes (comma-separated):')

Textarea(value='', description='Enter Altitudes (comma-separated):', placeholder='e.g., 0, 1000, 5000')

Checkbox(value=False, description='Show as Table?')

Button(description='Predict for Multiple Altitudes', style=ButtonStyle())

Output()

Output()